# Intro

The chat models are very different from the other models. We should spend more time for the data, especially during training. 

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

chat = [
    {"role": "user", "content": "Hello, how are you?"},
    {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
    {"role": "user", "content": "I'd like to show off how chat templating works!"},
]

tokenizer.apply_chat_template(chat, tokenize=False)

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

"<s>[INST] Hello, how are you? [/INST]I'm doing great. How can I help you today?</s> [INST] I'd like to show off how chat templating works! [/INST]"

In [4]:
# each model has different tokenizer! We can access the chat templates for each model
tokenizer.chat_template

"{{ bos_token }}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if message['role'] == 'user' %}{{ '[INST] ' + message['content'] + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ message['content'] + eos_token + ' ' }}{% else %}{{ raise_exception('Only user and assistant roles are supported!') }}{% endif %}{% endfor %}"

In [ ]:
# During training, we should also use the chat templates to rearrange the dataset
from transformers import AutoTokenizer
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")

chat1 = [
    {"role": "user", "content": "Which is bigger, the moon or the sun?"},
    {"role": "assistant", "content": "The sun."},
]
chat2 = [
    {"role": "user", "content": "Which is bigger, a virus or a bacterium?"},
    {"role": "assistant", "content": "A bacterium."},
]

dataset = Dataset.from_dict({"chat": [chat1, chat2]})
dataset = dataset.map(
    lambda x: {
        "formatted_chat": tokenizer.apply_chat_template(
            x["chat"], tokenize=False, add_generation_prompt=False
        )
    }
)
print(dataset["formatted_chat"][0])

## Fill in middle model

We can also fill the missing tokens in the middle of the sentence. We just need to tell the language model the content before and after the missing part. And the model will handle the missing part automatically.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "stabilityai/stable-code-3b", trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    "stabilityai/stable-code-3b",
    trust_remote_code=True,
    torch_dtype="auto",
    #   attn_implementation="flash_attention_2",
)
model.cuda()

# <fim_prefix> is the prefix code before the missiong part
# <fim_suffix> is the suffix code after the missiong part
# <fim_middle> is the missing part, and we add the token in the final part so that the model can predict it
inputs = tokenizer(
    "<fim_prefix>def fib(n):<fim_suffix>    else:\n        return fib(n - 2) + fib(n - 1)<fim_middle>",
    return_tensors="pt",
).to(model.device)
tokens = model.generate(
    **inputs,
    max_new_tokens=48,
    temperature=0.2,
    do_sample=True,
)
print(tokenizer.decode(tokens[0], skip_special_tokens=True))